# CPSC 585 Artificial Neural Networks Project 3
## Handwritten Character Recognition with a Convolutional Neural Network

### Group Members: 
### Jeremy Rico; jjrico@csu.fullerton.edu
### Daniel Walsh; danielwalsh27@csu.fullerton.edu
### Haojie Pan; haojie@csu.fullerton.edu

This program uses as a Convolutional Artifical Neural Network with one hidden layer to predict the identity of handwritten characters of all 26 letters of the english alphabet.

Dataset: EMNIST Letters - contains 145,600 28x28px images of handwritten characters divided into 26 classes. The dataset is divided into 108,000 training samples, 20,800 validation samples, and 20,800 test samples. More information about the dataset can be found here: https://www.nist.gov/itl/products-and-services/emnist-dataset

Our project utilizes the tf.keras module of TensorFlow to create, train, and test the neural network.

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
from scipy import io as sio
from google.colab import drive
#from keras.utils.np_utils import to_categorical

In [0]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

# inorder to import datasets you must have tensorflow datasets 
# installed by running the dollowing command in the terminal:
# pip install -q tensorflow tensorflow-datasets matplotlib
import tensorflow_datasets as tfds

In [16]:
# Define some vars we will use to construct the model
num_classes = 26
batch_size = 2000
epochs = 2000
neurons = 1000
img_rows, img_cols = 28, 28

#drive.mount('/content/drive')
mat_contents = sio.loadmat('/emnist-letters.mat')
data = mat_contents['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

y_train -= 1
y_val -= 1
y_test -= 1

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], "training samples")
print(X_val.shape[0], "validation samples")
print(X_test.shape[0], "test samples")
print(X_val.shape, y_val.shape)

104000 training samples
20800 validation samples
20800 test samples
(20800, 28, 28, 1) (20800, 26)


In [0]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(32, kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

In [18]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data=(X_val, y_val))

Epoch 1/2000
52/52 [==============================] - 6s 111ms/step - loss: 41.5345 - accuracy: 0.0422 - val_loss: 19.4863 - val_accuracy: 0.0466
Epoch 2/2000
52/52 [==============================] - 5s 105ms/step - loss: 34.5687 - accuracy: 0.0451 - val_loss: 15.2352 - val_accuracy: 0.0531
Epoch 3/2000
52/52 [==============================] - 5s 105ms/step - loss: 28.9512 - accuracy: 0.0479 - val_loss: 12.2022 - val_accuracy: 0.0606
Epoch 4/2000
52/52 [==============================] - 5s 105ms/step - loss: 24.2763 - accuracy: 0.0506 - val_loss: 9.8541 - val_accuracy: 0.0656
Epoch 5/2000
52/52 [==============================] - 5s 105ms/step - loss: 20.0752 - accuracy: 0.0538 - val_loss: 7.9467 - val_accuracy: 0.0698
Epoch 6/2000
52/52 [==============================] - 5s 105ms/step - loss: 16.5591 - accuracy: 0.0553 - val_loss: 6.4006 - val_accuracy: 0.0716
Epoch 7/2000
52/52 [==============================] - 5s 105ms/step - loss: 13.5550 - accuracy: 0.0565 - val_loss: 5.2185 - val

In [19]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Test loss: 0.27831992506980896
Test accuracy: 0.9127404093742371


In [20]:
## Test Results

| Epochs | Batch Size | Hidden Layer| Accuracy | Rows/Col  | Kernel Size | Pool Size |
|--------|------------|-------------|----------|-----------|-------------|-----------|
|5       |2000        |1000         |07.89%    | 28 x 28   |   3 x 3     |  2 x 2    | 
|5       |2000        |1000         |07.15%    | 28 x 28   |   3 x 3     |  2 x 2    | 
|500     |2000        |1000         |54.70%    | 28 x 28   |   3 x 3     |  2 x 2    | 
|700     |2000        |1000         |84.38%    | 28 x 28   |   3 x 3     |  2 x 2    | 
|1000    |2000        |1000         |88.52%    | 28 x 28   |   3 x 3     |  2 x 2    |
|1100    |2000        |1000         |89.08%    | 28 x 28   |   3 x 3     |  2 x 2    |
|1500    |2000        |1000         |90.96%    | 28 x 28   |   3 x 3     |  2 x 2    |
|2000    |2000        |1000         |91.27%    | 28 x 28   |   3 x 3     |  2 x 2    |
|        |            |             |          |           |             |           |
|        |            |             |          |           |             |           |
|        |            |             |          |           |             |           |
|        |            |             |          |           |             |           |

SyntaxError: ignored